In [1]:
import pickle
import re

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/UKB/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

print(list(demo.keys()))

['f_22001_0_0', 'f_21003_3_0', 'f_21003_2_0', 'f_21000_0_0', 'CaAfCarDiff', 'CaIndDiff', 'CaChiDiff', 'f_21000_0_0_comb', 'longitudinal']


In [2]:
import pickle

field = 25753

basedir = '/run/media/anton/AntonData1/UKB_MRI_data_11092022/'

fc2 = pickle.load(open(f'{basedir}/{field}_2.pkl', 'rb'))
fc3 = pickle.load(open(f'{basedir}/{field}_3.pkl', 'rb'))

print(len(fc2), len(fc3))

40471 2896


In [3]:
import numpy as np

first = []
second = []

for sub in demo['f_21003_3_0']:
    if sub not in demo['f_21003_2_0']:
        continue
    try:
        p0 = np.load(f'{demodir}/fc/{sub}_task-20227-2_fc.npy')
        p1 = np.load(f'{demodir}/fc/{sub}_task-20227-3_fc.npy')
#         p0 = fc2[sub]
#         p1 = fc3[sub]
        first.append(p0)
        second.append(p1)
    except:
        continue
    
first = np.stack(first)
second = np.stack(second)

print(len(first), len(second))

2722 2722


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
#         self.fc1 = nn.Linear(34716,10).float().cuda()
#         self.fc2 = nn.Linear(34716,10).float().cuda()
        self.fc3 = nn.Linear(34716,10).float().cuda()
        self.fc4 = nn.Linear(10,2).float().cuda()
        
    def forward(self, a, b, ba):
#         a = F.relu(self.fc1(a)).squeeze()
#         b = F.relu(self.fc2(b)).squeeze()
        ba = F.relu(self.fc3(ba)).squeeze()
#         x = torch.cat([a, b, ba], dim=1)
        x = self.fc4(ba)
        return x
    
N = 1300
idcs = np.random.permutation(len(first))
    
y = np.concatenate([np.zeros(N), np.ones(N)])

a = np.concatenate([first[idcs[:N]], second[idcs[N:N*2]]])
b = np.concatenate([second[idcs[:N]], first[idcs[N:N*2]]])
ba = b-a

atr, at, btr, bt, batr, bat, ytr, yt = train_test_split(a, b, ba, y, train_size=0.8)
atr, at, btr, bt, batr, bat = [torch.from_numpy(x).float().cuda() for x in [atr, at, btr, bt, batr, bat]]

ytr = torch.from_numpy(ytr).long().cuda() 
yt = torch.from_numpy(yt).long().cuda()
    
mlp = MLP()
optim = torch.optim.Adam(mlp.parameters(), lr=1e-4, weight_decay=1e-4)
ce = nn.CrossEntropyLoss()

nepochs = 2000
pperiod = 100

for e in range(nepochs):
    optim.zero_grad()
    yhat = mlp(atr, btr, batr)
    loss = ce(yhat, ytr)
    loss.backward()
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {float(loss)}')
        
print('Done')

with torch.no_grad():
    yhat = mlp(at, bt, bat)
    loss = torch.mean((torch.argmax(yhat, dim=1).long() == yt).float())
    print(loss)

0 0.7271744012832642
100 0.1502229869365692
200 0.0651685893535614
300 0.038126390427351
400 0.026704229414463043
500 0.021200716495513916
600 0.017650533467531204
700 0.015330527909100056
800 0.013998952694237232
900 0.01115129329264164
1000 0.009940357878804207
1100 0.009317653253674507
1200 0.008915882557630539
1300 0.008620425127446651
1400 0.008389702066779137
1500 0.008208883926272392
1600 0.008063566870987415
1700 0.007943530566990376
1800 0.007844584994018078
1900 0.007760337553918362
1999 0.007690546102821827
Done
tensor(0.8231, device='cuda:0')
